In [1]:
import sys 
sys.path.append('../../src/anomaly_detection_spatial_temporal_data/')

In [6]:
import os
import time
import json
from typing import List, Union, Optional, Tuple, Dict

from pathlib import Path, PosixPath

import pandas as pd
import numpy as np
import itertools

import re

import torch
from torch import nn

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import ncad
from ncad.ts import TimeSeries, TimeSeriesDataset
from ncad.ts import transforms as tr
from ncad.model import NCAD, NCADDataModule

import tqdm
import yaml

# Load model params

In [7]:
model_config_file = "../../conf/base/parameters/ncad.yml"

In [8]:
with open(model_config_file, "r") as stream:
    try:
        model_config = yaml.safe_load(stream)
        print(model_config)
    except yaml.YAMLError as exc:
        print(exc)

{'model_dir': 'data/07_model_output/ncad/', 'log_dir': 'data/07_model_output/ncad/logs/', 'evaluation_result_path': 'data/07_model_output/ncad/evaluation', 'exp_name': 'batadal-ncad', 'epochs': 3, 'limit_val_batches': 1.0, 'num_sanity_val_steps': 1, 'injection_method': 'local_outliers', 'ratio_injected_spikes': 0.1, 'window_length': 500, 'suspect_window_length': 10, 'num_series_in_train_batch': 1, 'num_crops_per_series': 32, 'num_workers_loader': 0, 'tcn_kernel_size': 7, 'tcn_layers': 1, 'tcn_out_channels': 16, 'tcn_maxpool_out_channels': 32, 'embedding_rep_dim': 64, 'normalize_embedding': True, 'distance': 'cosine', 'classifier_threshold': 0.5, 'threshold_grid_length_val': 0.1, 'threshold_grid_length_test': 0.05, 'coe_rate': 0.5, 'mixup_rate': 2.0, 'learning_rate': 0.0001, 'check_val_every_n_epoch': 25, 'stride_roll_pred_val_test': 5, 'val_labels_adj': True, 'test_labels_adj': True, 'max_windows_unfold_batch': 5000}
{'model_dir': 'data/07_model_output/ncad/', 'log_dir': 'data/07_model

# Load data

Load processed CSVs into `TimeSeries` object

In [9]:
def load_dataset(train_clean: pd.DataFrame, train_anom: pd.DataFrame, test: pd.DataFrame, sensor_cols: List[str], label_col: str) -> Tuple:
    if type(sensor_cols) == str:
        sensor_cols = sensor_cols.split("\n")
    train_dataset = TimeSeriesDataset()
    test_dataset = TimeSeriesDataset()
    
    
    for sensor in sensor_cols:
        train_dataset.append(
            TimeSeries(
                values=train_clean[sensor].values,
                labels=None,
                item_id=f"{sensor}_train_clean"
            )
        )
        
        train_dataset.append(
            TimeSeries(
                values=train_anom[sensor].values,
                labels=train_anom[label_col].values,
                item_id=f"{sensor}_train_anom"
            )
        )
        
        test_dataset.append(
            TimeSeries(
                values=test[sensor].values,
                labels=test[label_col].values,
                item_id=f"{sensor}_test"
            )
        )
           
    return train_dataset, test_dataset

In [10]:
train_clean = pd.read_csv("../../data/03_primary/iot/iot_ncad_train.csv")
train_anom = pd.read_csv("../../data/03_primary/iot/iot_ncad_train_anom.csv")
test = pd.read_csv("../../data/03_primary/iot/iot_ncad_test.csv")

with open(f"../../data/03_primary/iot_sensor_list_batadal.txt", "r") as f:
    sensors = f.read().split("\n")

In [11]:
train_dataset, test_dataset = load_dataset(train_clean, train_anom, test, sensors, "label")

# Standardize data

substract median, divide by interquartile range

In [12]:
def standardize_dataset(train_set: TimeSeriesDataset, test_set: TimeSeriesDataset) -> Tuple:
    # Standardize TimeSeries values (substract median, divide by interquartile range)
    scaler = tr.TimeSeriesScaler(type="robust")
    train_set = TimeSeriesDataset(ncad.utils.take_n_cycle(scaler(train_set), len(train_set)))
    test_set = TimeSeriesDataset(ncad.utils.take_n_cycle(scaler(test_set), len(test_set)))
    ts_channels = train_set[0].shape[1]
    return train_set, test_set, ts_channels

In [13]:
train_standard, test_standard, ts_channels = standardize_dataset(train_dataset, test_dataset)

# Inject anomalies

NCAD is a semi-supervised algorithm, and includes functionalities to inject anomalies into the training data set

In [14]:
def batadal_inject_anomalies(
    dataset: TimeSeriesDataset,
    injection_method: str,
    ratio_injected_spikes: float,
) -> TimeSeriesDataset:

    # without this, kedro treats the output as a list of multiple elements rather than
    scratch = ""
    # dataset is transformed using a TimeSeriesTransform depending on the type of injection
    if injection_method == "None":
        return dataset
    elif injection_method == "local_outliers":
        if ratio_injected_spikes is None:
            # Inject synthetic anomalies: LocalOutlier
            ts_transform = tr.LocalOutlier(area_radius=500, num_spikes=360)
        else:
            ts_transform = tr.LocalOutlier(
                area_radius=700,
                num_spikes=ratio_injected_spikes,
                spike_multiplier_range=(1.0, 3.0),
                # direction_options = ['increase'],
            )

        # Generate many examples of injected time series
        multiplier = 20
        ts_transform_iterator = ts_transform(itertools.cycle(dataset))
        dataset_transformed = ncad.utils.take_n_cycle(
            ts_transform_iterator, multiplier * len(dataset)
        )
        dataset_transformed = TimeSeriesDataset(dataset_transformed)
    else:
        raise ValueError(f"injection_method = {injection_method} not supported!")

    return dataset_transformed

In [15]:
%%time
train_injected = batadal_inject_anomalies(
    train_standard, 
    model_config["injection_method"], 
    model_config["ratio_injected_spikes"]
)

CPU times: user 2min 44s, sys: 206 ms, total: 2min 45s
Wall time: 2min 45s
CPU times: user 2min 44s, sys: 206 ms, total: 2min 45s
Wall time: 2min 45s


# Split Test into val/test

In [16]:
def split_test_into_val(test_set: TimeSeriesDataset, window_length: int, suspect_window_length: int) -> Tuple:
    # Split test dataset in two, half for validation and half for test
    _, validation_set, test_set = ncad.ts.split_train_val_test(
        data=test_set,
        val_portion=0.5,  # No labels in training data that could serve for validation
        test_portion=0.5,  # This dataset provides test set, so we don't need to take from training data
        split_method="past_future_with_warmup",
        split_warmup_length=window_length - suspect_window_length,
        verbose=False,
    )
    
    return validation_set, test_set

In [17]:
validation_split, test_split = split_test_into_val(
    test_standard, 
    model_config["window_length"],
    model_config["suspect_window_length"]
)

# Construct data module

In [18]:
def construct_data_module(
    train_set_transformed: TimeSeriesDataset, validation_set: TimeSeriesDataset, test_set: TimeSeriesDataset,
    window_length: int, suspect_window_length: int, 
    num_series_in_train_batch: int,
    num_crops_per_series: int,
    stride_roll_pred_val_test: int,
    num_workers_loader: int
    
) -> NCADDataModule:
    # Define DataModule for training with lighting (window cropping + window injection)#
    data_module = NCADDataModule(
        train_ts_dataset=train_set_transformed,
        validation_ts_dataset=validation_set,
        test_ts_dataset=test_set,
        window_length=window_length,
        suspect_window_length=suspect_window_length,
        num_series_in_train_batch=num_series_in_train_batch,
        num_crops_per_series=num_crops_per_series,
        label_reduction_method="any",
        stride_val_and_test=stride_roll_pred_val_test,
        num_workers=num_workers_loader,
    )
    
    return data_module

In [19]:
ncad_data_module = construct_data_module(
    train_injected, validation_split, test_split,
    model_config["window_length"], model_config["suspect_window_length"],
    model_config["num_series_in_train_batch"],
    model_config["num_crops_per_series"],
    model_config["stride_roll_pred_val_test"],
    model_config["num_workers_loader"]
)

# Set up call backs

In [20]:
def set_up_callbacks(
    ## General
    model_dir: Union[str, PosixPath],
    log_dir: Union[str, PosixPath],
) -> ModelCheckpoint:
    # Experiment name #
    time_now = time.strftime("%Y-%m-%d-%H%M%S", time.localtime())
    exp_name = f"batadal-{time_now}"

    ### Training the model ###

    logger = TensorBoardLogger(save_dir=log_dir, name=exp_name)

    # Checkpoint callback, monitoring 'val_f1'
    checkpoint_cb = ModelCheckpoint(
        monitor="val_f1",
        dirpath=model_dir,
        filename="ncad-model-" + exp_name + "-{epoch:02d}-{val_f1:.4f}",
        save_top_k=1,
        mode="max",
    )
    
    return checkpoint_cb, logger, exp_name

In [21]:
checkpointer, tb_logger, exp_name = set_up_callbacks(
    model_config["model_dir"],
    model_config["log_dir"]
)

/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:360: UserWarning: Checkpoint directory data/07_model_output/ncad/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:360: UserWarning: Checkpoint directory data/07_model_output/ncad/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


# Train model

In [22]:
def set_and_train_model(
    data_module: NCADDataModule, 
    model_dir: Union[str, PosixPath],
    logger: TensorBoardLogger,
    epochs: int,
    limit_val_batches: float,
    num_sanity_val_steps: int,
    check_val_every_n_epoch: int,
    checkpoint_cb: ModelCheckpoint,
    ts_channels: int,
    window_length: int,
    suspect_window_length: int, 
    tcn_kernel_size: int,
    tcn_layers: int,
    tcn_out_channels: int,
    tcn_maxpool_out_channels: int,
    embedding_rep_dim: int,
    normalize_embedding: bool,
    # hpars for classifier
    distance: str,
    classifier_threshold: float,
    threshold_grid_length_test: float,
    # hpars for optimizer
    learning_rate: float,
    # hpars for anomalizers
    coe_rate: float,
    mixup_rate: float,
    # hpars for validation and test
    stride_roll_pred_val_test: int,
    val_labels_adj: bool,
    test_labels_adj: bool,
    max_windows_unfold_batch: Optional[int] = 5000,
    
    evaluation_result_path: Optional[Union[str, PosixPath]] = None,
) -> Tuple:
    if distance == "cosine":
        # For the contrastive approach, the cosine distance is used
        distance = ncad.model.distances.CosineDistance()
    elif distance == "L2":
        # For the contrastive approach, the L2 distance is used
        distance = ncad.model.distances.LpDistance(p=2)
    elif distance == "non-contrastive":
        # For the non-contrastive approach, the classifier is
        # a neural-net based on the embedding of the whole window
        distance = ncad.model.distances.BinaryOnX1(rep_dim=embedding_rep_dim, layers=1)

    # Instantiate model #
    model = NCAD(
        ts_channels=ts_channels,
        window_length=window_length,
        suspect_window_length=suspect_window_length,
        # hpars for encoder
        tcn_kernel_size=tcn_kernel_size,
        tcn_layers=tcn_layers,
        tcn_out_channels=tcn_out_channels,
        tcn_maxpool_out_channels=tcn_maxpool_out_channels,
        embedding_rep_dim=embedding_rep_dim,
        normalize_embedding=normalize_embedding,
        # hpars for classifier
        distance=distance,
        classification_loss=nn.BCELoss(),
        classifier_threshold=classifier_threshold,
        threshold_grid_length_test=threshold_grid_length_test,
        # hpars for anomalizers
        coe_rate=coe_rate,
        mixup_rate=mixup_rate,
        # hpars for validation and test
        stride_rolling_val_test=stride_roll_pred_val_test,
        val_labels_adj=val_labels_adj,
        test_labels_adj=test_labels_adj,
        max_windows_unfold_batch=max_windows_unfold_batch,
        # hpars for optimizer
        learning_rate=learning_rate,
    )
    
    trainer = Trainer(
        gpus=1 if torch.cuda.is_available() else 0,
        default_root_dir=model_dir,
        logger=logger,
        min_epochs=epochs,
        max_epochs=epochs,
        limit_val_batches=limit_val_batches,
        num_sanity_val_steps=1,
        check_val_every_n_epoch=check_val_every_n_epoch,
        callbacks=[checkpoint_cb],
        # callbacks=[checkpoint_cb, earlystop_cb, lr_logger],
        auto_lr_find=False,
    )
    
    trainer.fit(
        model=model,
        datamodule=data_module,
    )
    
    # Metrics on validation and test data #
    evaluation_result = trainer.test()
    evaluation_result = evaluation_result[0]
    classifier_threshold = evaluation_result["classifier_threshold"]
    
    # Save evaluation results
    if evaluation_result_path is not None:
        path = evaluation_result_path
        path = PosixPath(path).expanduser() if str(path).startswith("~") else Path(path)
        with open(path, "w") as f:
            json.dump(evaluation_result, f, cls=ncad.utils.NpEncoder)

    for key, value in evaluation_result.items():
        # if key.startswith('test_'):
        print(f"{key}={value}")
        
    return model_dir, classifier_threshold


In [23]:
_, classifier_threshold = set_and_train_model(
    ncad_data_module, 
    model_config["model_dir"],
    tb_logger,
    model_config["epochs"],
    model_config["limit_val_batches"],
    model_config["num_sanity_val_steps"],
    model_config["check_val_every_n_epoch"],
    checkpointer,
    ts_channels,
    model_config["window_length"],
    model_config["suspect_window_length"], 
    model_config["tcn_kernel_size"],
    model_config["tcn_layers"],
    model_config["tcn_out_channels"],
    model_config["tcn_maxpool_out_channels"],
    model_config["embedding_rep_dim"],
    model_config["normalize_embedding"],
    # hpars for classifier
    model_config["distance"],
    model_config["classifier_threshold"],
    model_config["threshold_grid_length_test"],
    # hpars for optimizer
    model_config["learning_rate"],
    # hpars for anomalizers
    model_config["coe_rate"],
    model_config["mixup_rate"],
    # hpars for validation and test
    model_config["stride_roll_pred_val_test"],
    model_config["val_labels_adj"],
    model_config["test_labels_adj"],
    model_config["max_windows_unfold_batch"],
    
    model_config["evaluation_result_path"],
)

/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Metric` was deprecated since v1.3.0 in favor of `torchmetrics.metric.Metric`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/src/anomaly_detection_spatial_temporal_data/model/ncad/src/ncad/utils/pl_metrics.py:29: UserWarning: Metric `CachePredictions` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  "Metric `CachePredictions` will save all targets and predictions in buffer."
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The 

Validation sanity check: 100%|██████████| 1/1 [00:00<00:00,  5.71it/s]

/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'
/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be

/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


Epoch 2: 100%|██████████| 1720/1720 [01:01<00:00, 27.96it/s, loss=2.39, v_num=0, classifier_threshold=0.000, val_f1=0.375, train_loss_step=0.654] 


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


Testing:  23%|██▎       | 10/43 [00:00<00:00, 94.36it/s]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/kedro-ncad-venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


Testing: 100%|██████████| 43/43 [00:04<00:00,  9.03it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'classifier_threshold': 0.6,
 'test_FN': 5260,
 'test_FP': 5455,
 'test_TN': 30450,
 'test_TP': 3770,
 'test_f0.5': 0.4104071413019813,
 'test_f1': 0.4130375239660367,
 'test_f2': 0.4157018414378652,
 'test_precision': 0.4086720867208672,
 'test_recall': 0.4174972314507198}
--------------------------------------------------------------------------------
classifier_threshold=0.6
test_f1=0.4130375239660367
test_f2=0.4157018414378652
test_f0.5=0.4104071413019813
test_precision=0.4086720867208672
test_recall=0.4174972314507198
test_TN=30450
test_FN=5260
test_TP=3770
test_FP=5455
Testing: 100%|██████████| 43/43 [00:04<00:00,  9.03it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'classifier_threshold': 0.6,
 'test_FN': 5260,
 'test_FP': 5455,
 'test_TN': 30450,
 'te

# Perform inference

In [24]:
def load_and_evaluate(
    model_dir: Union[str, PosixPath],
    exp_name: str,
    data_set: TimeSeriesDataset,
    stride: int,
    classifier_threshold: float
):
    model_dir = PosixPath(model_dir).expanduser() if str(model_dir).startswith("~") else Path(model_dir)
    
    # Load top performing checkpoint
    # ckpt_path = [x for x in model_dir.glob('*.ckpt')][-1]
    ckpt_file = [
        file
        for file in os.listdir(model_dir)
        if (file.endswith(".ckpt") and file.startswith("ncad-model-" + exp_name))
    ]

    ckpt_file = ckpt_file[-1]
    ckpt_path = model_dir / ckpt_file
    model = NCAD.load_from_checkpoint(ckpt_path)

    anomaly_probs_avg, anomaly_vote = model.tsdetect(data_set, stride, classifier_threshold)
    
    print(f"NCAD on batadal dataset finished successfully!")    
    
    return anomaly_probs_avg, anomaly_vote

In [25]:
anomaly_probs_avg, anomaly_vote = load_and_evaluate(
    model_config["model_dir"],
    exp_name,
    test_standard,
    model_config["stride_roll_pred_val_test"],
    classifier_threshold
)

data/07_model_output/ncad 1 ['ncad-model-batadal-2022-07-13-214807-epoch=02-val_f1=0.3752.ckpt']
data/07_model_output/ncad 1 ['ncad-model-batadal-2022-07-13-214807-epoch=02-val_f1=0.3752.ckpt']


/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/src/anomaly_detection_spatial_temporal_data/model/ncad/src/ncad/utils/pl_metrics.py:29: UserWarning: Metric `CachePredictions` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  "Metric `CachePredictions` will save all targets and predictions in buffer."
/home/ec2-user/SageMaker/repos/anomaly-detection-spatial-temporal-data/src/anomaly_detection_spatial_temporal_data/model/ncad/src/ncad/utils/pl_metrics.py:29: UserWarning: Metric `CachePredictions` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  "Metric `CachePredictions` will save all targets and predictions in buffer."


(43, 2089) (43, 2089)
NCAD on batadal dataset finished successfully!
(43, 2089) (43, 2089)
NCAD on batadal dataset finished successfully!


In [26]:
print(anomaly_probs_avg.shape, anomaly_vote.shape)

(43, 2089) (43, 2089)
(43, 2089) (43, 2089)


# References
Riccardo Taormina and Stefano Galelli and Nils Ole Tippenhauer and Elad Salomons and Avi Ostfeld and Demetrios G. Eliades and Mohsen Aghashahi and Raanju Sundararajan and Mohsen Pourahmadi and M. Katherine Banks and B. M. Brentan and Enrique Campbell and G. Lima and D. Manzi and D. Ayala-Cabrera and M. Herrera and I. Montalvo and J. Izquierdo and E. Luvizotto and Sarin E. Chandy and Amin Rasekh and Zachary A. Barker and Bruce Campbell and M. Ehsan Shafiee and Marcio Giacomoni and Nikolaos Gatsis and Ahmad Taha and Ahmed A. Abokifa and Kelsey Haddad and Cynthia S. Lo and Pratim Biswas and M. Fayzul K. Pasha and Bijay Kc and Saravanakumar Lakshmanan Somasundaram and Mashor Housh and Ziv Ohar; "The Battle Of The Attack Detection Algorithms: Disclosing Cyber Attacks On Water Distribution Networks." Journal of Water Resources Planning and Management, 144 (8), August 2018

Chris U. Carmona, François-Xavier Aubet, Valentin Flunkert, and Jan Gasthaus. 2021. Neural Contextual Anomaly Detection for Time Series.